In [9]:
# Baseline Model on the Sonar Dataset
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataframe = read_csv("../datasets/sonar.all.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

## Without dropout

In [6]:
# baseline
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' ))
    # Compile model
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss= 'binary_crossentropy' , optimizer=sgd, metrics=[ 'accuracy' ])
    return model

In [7]:
numpy.random.seed(seed)
estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_baseline, epochs=300,batch_size=16, verbose=0)))

In [10]:
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 86.04% (4.58%)


## With dropout in input layer

In [17]:
# dropout in the input layer with weight constraint
def create_model_with_dropout():
    # create model
    model = Sequential()
    #create a new dropout layer between input and first hidden layer with 20% of rate
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, kernel_initializer= 'normal' , activation= 'relu' ,
    kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' ,
    kernel_constraint=maxnorm(3)))
    model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' ))
    # Compile model
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss= 'binary_crossentropy' , optimizer=sgd, metrics=[ 'accuracy' ])
    return model

In [18]:
numpy.random.seed(seed)
estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_model_with_dropout, epochs=300,batch_size=16, verbose=0)))

In [19]:
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 82.12% (7.62%)


## With dropout on hidden layer

In [20]:
def create_model_with_dropout_on_hidden():
    # create model
    model = Sequential()       
    model.add(Dense(60,input_dim=60, kernel_initializer= 'normal' , activation= 'relu' ,
    kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' ,
    kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' ))
    # Compile model
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss= 'binary_crossentropy' , optimizer=sgd, metrics=[ 'accuracy' ])
    return model

In [21]:
numpy.random.seed(seed)
estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_model_with_dropout_on_hidden, epochs=300,batch_size=16, verbose=0)))

In [22]:
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 83.59% (5.06%)
